In [1]:
from sklearn.model_selection import train_test_split
from metric.calculate import calculate
import pickle
from models.linear import SVM, LinearSVM, LogisticReg
from models.tree import CatBoost, RandomForest, GradientBoosting
from models.baseline import Baseline
import matplotlib.pyplot as plt
from collections import Counter
import warnings
from utils.usage_loader import initial_feature_names
from utils.encoder import encode_column
from sklearn.utils import shuffle
import numpy as np

In [2]:
with open('data/usage_loader.pickle', 'rb') as f:
    usage_loader = pickle.load(f)

In [3]:
usages = list(filter(lambda x: 'java.lang.Override' not in x.annotation_name, usage_loader.load_all()))
train_usages = []
test_usages = []
for usage in usages:
    if 'java/idea-ui' in usage.file_path:
        test_usages.append(usage)
    else:
        train_usages.append(usage)
train_usages = shuffle(train_usages, random_state=123)[:10000]
usages = train_usages + test_usages
raw_X = np.array([np.array(usage.features_list, dtype=object) for usage in usages])
X = None
all_new_names = []
if len(raw_X) == 0:
    X = np.array([])
    all_new_names = initial_feature_names
else:
    for col in range(raw_X.shape[1]):
        new_columns, new_names = encode_column(raw_X[:, col], len(train_usages),
                                               initial_feature_names[col], 100)
        if new_columns is None:
            continue
        all_new_names += new_names
        if X is None:
            X = new_columns
        else:
            X = np.concatenate((X, new_columns), axis=1)
# if need_polynomial and len(X) > 0:
#     sel = VarianceThreshold(threshold=0.01)
#     print(len(X[0]))
#     X = sel.fit_transform(X)
#     print(len(X[0]))
#     poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
#     X = poly.fit_transform(X)
#     print(len(X[0]))
#     sel = VarianceThreshold(threshold=0.01)
#     X = sel.fit_transform(X)
#     print(len(X[0]))
y = np.array([usage.annotation_name for usage in usages])

In [4]:
len(X)

13238

In [5]:
len(y)

13238

In [6]:
len(test_usages)

3238

In [7]:
from metric.calculate import calculate_and_print

In [8]:
X_train = X[:10000]
y_train = y[:10000]
X_test = X[10000:]
y_test = y[10000:]

In [10]:
calculate_and_print(X_train, X_test, y_train, y_test, Baseline())
calculate_and_print(X_train, X_test, y_train, y_test, CatBoost())

Count: 3238
Top 1: 0.6553428042001236
Top 2: 0.8638048177887585
Top 3: 0.8638048177887585
Top 4: 0.8866584311303274
Top 5: 0.8885114268066708
Top1 1: 0.6738727609635577
Mean: 3.816244595429277
Count: 3238
Top 1: 0.7161828289067326
Top 2: 0.9104385423100679
Top 3: 0.9298949969116739
Top 4: 0.9478072884496603
Top 5: 0.9567634342186535
Top1 1: 0.7402717726991971
Mean: 2.547869054972205
